In [1]:
#!/usr/bin/python3
'''*.csv Data Format: ['Case', 'RunTime', 'Object', 'EnterTime', 'Velocity', 'Acceleration']'''
import csv, os, numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt

In [2]:
def get_data(fullpath) -> pd.DataFrame:
  df = pd.read_csv(fullpath)
  return df

In [3]:
def find_local_maximums(df: pd.DataFrame, var_name1:'Case', var_name2: 'Velocity') -> pd.DataFrame:
  df_grp = df.groupby(var_name1)
  idx = []
  for key, key_df in df_grp:
    pre_pre_idx = None
    pre_pre_var = None
    pre_idx = None
    pre_var = None
    pre_left = False
    pre_right = False
    for index, row in key_df.iterrows():
      if pre_pre_idx is None:
        pre_pre_idx = index
        pre_pre_var = row[var_name2]
      elif pre_idx is None:
        pre_idx = index
        pre_var = row[var_name2]
      else:
        cur_var = row[var_name2]
        # previous
        pre_left = (pre_pre_var <= pre_var)
        pre_right = (cur_var <= pre_var)
        if pre_left and pre_right:
          idx.append(pre_idx)
        # update
        pre_pre_idx = pre_idx
        pre_pre_var = pre_var
        pre_idx = index
        pre_var = cur_var
  return idx

In [4]:
def find_local_minimums(df: pd.DataFrame, var_name1:'Case', var_name2: 'fastcity') -> pd.DataFrame:
  df_grp = df.groupby(var_name1)
  idx = []
  for key, key_df in df_grp:
    pre_pre_idx = None
    pre_pre_var = None
    pre_idx = None
    pre_var = None
    pre_left = False
    pre_right = False
    for index, row in key_df.iterrows():
      if pre_pre_idx is None:
        pre_pre_idx = index
        pre_pre_var = row[var_name2]
      elif pre_idx is None:
        pre_idx = index
        pre_var = row[var_name2]
      else:
        cur_var = row[var_name2]
        # previous
        pre_left = (pre_pre_var >= pre_var)
        pre_right = (cur_var >= pre_var)
        if pre_left and pre_right:
          idx.append(pre_idx)
        # update
        pre_pre_idx = pre_idx
        pre_pre_var = pre_var
        pre_idx = index
        pre_var = cur_var
  return idx

In [5]:
def get_quantile_and_IQR(df: pd.DataFrame) -> list:
  # shape (2,1) -> squeeze() -> shape (2,)
    [q1, q3] = df.quantile([.25, .75]).squeeze()
    #print(f"{[q1,q3]}")
    IQR = q3-q1
    return [q1, q3, IQR]

In [6]:
def remove_outliers(df: pd.DataFrame, col_name, by_case) -> pd.DataFrame:
  '''Input: df, target variable's column name, if the removement is by case
     Output: df without outlier data
     Outliers Calculation: data points < (q1-1.5*IQR) or data points > (q3 + 1.5*IQR)
  '''
  if by_case:
   df_by_case = df.groupby('Case')
   df_out = pd.DataFrame(columns=df.columns)
   for key, key_df in df_by_case:
      [q1, q3, IQR] = get_quantile_and_IQR(key_df[col_name])
      temp = key_df[(key_df[col_name] >= q1-1.5*IQR) & (key_df[col_name] <= q3+1.5*IQR)]
      df_out = pd.concat([df_out, temp])
   return df_out
  else:
   [q1, q3, IQR] = get_quantile_and_IQR(df[col_name])
   return df[(df[col_name] >= q1-1.5*IQR) & (df[col_name] <= q3+1.5*IQR)]

In [7]:
'''
1. Get the data of the inner button's velocity or acceleration
2. Delete the outliers and perhaps the positive acceleration
3. Calculate the mean
'''
def get_no_outlier_for_var2(file_path, object_name, var_name1:'RunTime', var_name2: 'Velocity or Acceleration') -> pd.DataFrame:
  # Read one csv and extract data of a specific object
  df_one = get_data(file_path)
  if (var_name2 == "Acceleration"):
    df_one = df_one[(df_one[var_name2] < 0)]
  df_vars = df_one[[var_name1,var_name2]]
  df_no_outliers = remove_outliers(df_vars)
  return df_no_outliers
  

In [8]:
def get_biggest_EnterTime(file_path, var_name1:'Case', var_name2: 'EnterTime'):
  df_one = get_data(file_path)
  df_vars = df_one[[var_name1,var_name2]]
  idx = df_vars.groupby(var_name1)[var_name2].idxmax()
  return df_vars.loc[idx,[var_name1,var_name2]]

In [9]:
def get_crest_for_var2(file_path, var_name1:'Case', var_name2: 'Velocity or Acceleration', AddRunTime)->pd.DataFrame:
  df = get_data(file_path)
  df_one = df[[var_name1, var_name2]]
  print(df_one.shape)
  df_one_no_out = remove_outliers(df_one)
  print(df_one_no_out.shape)
  idx = find_local_minimums(df_one_no_out, var_name1, var_name2)
  if AddRunTime:
    df_one = df[[var_name1, 'RunTime', var_name2]]
    df_crest = df_one.loc[idx, [var_name1, 'RunTime', var_name2]]
  else:
    df_one = df[[var_name1, var_name2]]
    df_crest = df_one.loc[idx, [var_name1, var_name2]]
  return df_crest

In [10]:
def get_peak_for_var2(file_path, var_name1:'Case', var_name2: 'Velocity or Acceleration', AddRunTime)->pd.DataFrame:
  df_one = get_data(file_path)
  idx = find_local_maximums(df_one, var_name1, var_name2)
  if AddRunTime:
    df_one = df_one[[var_name1, 'RunTime', var_name2]]
    df_crest = df_one.loc[idx, [var_name1, 'RunTime', var_name2]]
  else:
    df_one = df_one[[var_name1, var_name2]]
    df_crest = df_one.loc[idx, [var_name1, var_name2]]
  return df_crest

In [11]:
def lineplot_all_and_crest(x:str, y:str, df_all: pd.DataFrame, df_crest: pd.DataFrame, save_path):
  df_all.insert(df_all.shape[1], 'Class', 'all')
  df_crest.insert(df_crest.shape[1], 'Class', 'crest')
  df_concat = pd.concat([df_all,df_crest])
  fig = sns.lineplot(x=x, y=y, hue='Class', data=df_concat)
  figure = fig.get_figure()
  figure.savefig(save_path, dpi=400)
  

In [12]:
def describe_by_Case(df: pd.DataFrame, col_name) -> pd.DataFrame:
  df_case = pd.DataFrame(columns=['Number','Case','Min','Mean','Max'])
  for key, key_df in df:
    df_case.loc[len(df_case.index)] = [key[0],key[1],key_df[col_name].min(),key_df[col_name].mean(),key_df[col_name].max()]
  return df_case

In [13]:
import os
File_Dir = r"C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1"
os.chdir(File_Dir)


In [22]:
analyse_date="230426 - "
for folder_name in os.listdir(File_Dir):
    if folder_name.endswith("Cancel"):
        pass
    else:
        folder_num = int(folder_name)
        if 1 <= folder_num <= 10:
            new_name = f"{analyse_date}{folder_name} - Cancel"
            os.rename(folder_name, new_name)
        
for folder in os.listdir(File_Dir):
    if folder == new_name:
        print("Success")

ValueError: invalid literal for int() with base 10: '1 - Execute'

In [24]:
import pandas as pd
item_dir = r"C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute"
output_dir = r"C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Cancel"
dfs = []
file_output_paths =[]

#Get all files from separated folders. 
for item in os.listdir(item_dir):
    #print(item)
    new_path = item_dir + "\\" + item
    print(new_path)
    for i in os.listdir(new_path):
        print(i)        
        new_file_path = new_path + "\\" + i
        file_output_path = output_dir + "\\" + i
        file_output_paths.append(file_output_path)
        df = pd.read_csv(new_file_path)
        
        dfs.append(df)
#print(dfs)
print(file_output_paths)
        #for case in df["Case"]:
            #if case >= 10:
                #print(case)

C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute\1 - Execute
20221105-06-53-56-AccelerationIn.csv
C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute\10 - Execute
20230422-11-28-31-AccelerationIn.csv
C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute\2 - Execute
20221105-06-59-41-AccelerationIn.csv
C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute\3 - Execute
20221105-07-04-37-AccelerationIn.csv
C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute\4 - Execute
20221105-07-11-48-AccelerationIn.csv
C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute\5 - Execute
20221105-07-15-12-AccelerationIn.csv
C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute\6 - Execute
20221105-07-20-51-AccelerationIn.csv
C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute\7 - Execute
20221105-07-27-57-AccelerationIn.csv

In [145]:
dfs

[     Case  RunTime     Object  EnterTime  Velocity  Acceleration Condition  \
 0       1     6.60  FastTapIn       0.00  0.148254      2.601382             
 1       1     6.62  FastTapIn       0.02  0.074973     -3.664041             
 2       1     6.64  FastTapIn       0.04  0.084562      0.479427             
 3       1     6.66  FastTapIn       0.06  0.029872     -2.734504             
 4       1     6.68  FastTapIn       0.08  0.059273      1.470081             
 ..    ...      ...        ...        ...       ...           ...       ...   
 428    30    64.24  FastTapIn       0.04  0.519791      3.887051             
 429    30    64.26  FastTapIn       0.06  0.786858     13.353390             
 430    30    64.28  FastTapIn       0.08  0.289721    -24.856850             
 431    30    64.30  FastTapIn       0.10  0.468788      8.953322             
 432    30    64.32  FastTapIn       0.12  0.218583    -12.510240             
 
      PrepTime  
 0           0  
 1           0  

In [167]:
new_dfs = []
for filepath in file_output_paths:
    print(filepath)
    for df in dfs:
        df = df[df["Case"] >= 10]
        new_dfs.append(df)
        print(new_dfs)
        for df in new_dfs:
            df.to_csv(filepath, index = False)

C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute_only\20221105-06-53-56-AccelerationIn.csv
C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute_only\20230422-11-28-31-AccelerationIn.csv
C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute_only\20221105-06-59-41-AccelerationIn.csv
C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute_only\20221105-07-04-37-AccelerationIn.csv
C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute_only\20221105-07-11-48-AccelerationIn.csv
C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute_only\20221105-07-15-12-AccelerationIn.csv
C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute_only\20221105-07-20-51-AccelerationIn.csv
C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute_only\20221105-07-27-57-AccelerationIn.csv
C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 2304

In [26]:
for i, filepath in enumerate(file_output_paths):
    df = dfs[i]
    df = df[df["Case"] >= 10]
    df.to_csv(filepath, index=False)

In [156]:
# This block is for checking if datas are correct.

Temp_dir = r"C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Execute_only"
for item in os.listdir(Temp_dir):
    path = Temp_dir + "\\" + item
    #print(path)
    df = pd.read_csv(path)
    print(df)

     Case  RunTime     Object  EnterTime  Velocity  Acceleration Condition  \
0       1     3.72  FastTapIn       0.00  1.052463     25.543160             
1       1     3.74  FastTapIn       0.02  0.405517    -32.347290             
2       1     3.76  FastTapIn       0.04  0.397746     -0.388578             
3       1     3.78  FastTapIn       0.06  0.340581     -2.858217             
4       1     3.80  FastTapIn       0.08  0.097638    -12.147160             
..    ...      ...        ...        ...       ...           ...       ...   
430    10    18.54  FastTapIn       0.78  0.196512      5.948477             
431    10    18.56  FastTapIn       0.80  0.117899     -3.930622             
432    10    18.58  FastTapIn       0.82  0.464636     17.336850             
433    10    18.60  FastTapIn       0.84  0.223316    -12.066030             
434    10    18.62  FastTapIn       0.86  0.453317     11.500090             

     PrepTime  
0           0  
1           0  
2           0  

In [7]:
Analyse_date = "230426 - "
folder_suffix =  " - Cancel"
dirs = r"C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Cancel"
for folder in os.listdir(dirs):
    #print(folder)
    for i in range(1,11):
        Folder_name = f"{Analyse_date}{i}{folder_suffix}"
        
        dir_path = os.path.join(dirs, Folder_name)
        print(dir_path)
        #subfolderPath = os.path.join(dir_path, Folder_name)
        #print(subfolderPath)
        if not(os.path.exists(dir_path)):
            os.makedirs(dir_path)
            print(f"{dir_path} created")
        else:
            print(f"{subfolderPath} exists")

In [234]:
Data_source = r"C:\Users\angel\Desktop\S1DA\BacktraceDataset"
stay_csvs = []
backtrace_csvs = []
for folder in os.listdir(Data_source):
    folder_path = f"\\{folder}"
    file_path = f"{Data_source}{folder_path}"
    for file in os.listdir(file_path):
        data_directory_paths = file_path+ "\\"+ file
        if data_directory_paths.endswith(".DS_Store"):
            os.remove(data_directory_paths)
        else:
             for dirs in os.listdir(data_directory_paths):
                    dirs_loc = data_directory_paths + "\\" + dirs
                    if data_directory_paths.endswith("backtrace"):
                        backtrace_csvs.append(dirs_loc)
                    elif data_directory_paths.endswith("stay"):
                        stay_csvs.append(dirs_loc)
print(backtrace_csvs)
print(stay_csvs)
                    
                    
        #if data_directory.endswith(".DS_Store"):
            
        


['C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\1\\backtrace\\20221105-06-53-56-FastTapIn.csv', 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\10\\backtrace\\20230422-11-28-31-FastTapIn.csv', 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\2\\backtrace\\20221105-06-59-41-FastTapIn.csv', 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\3\\backtrace\\20221105-07-04-37-FastTapIn.csv', 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\4\\backtrace\\20221105-07-11-48-FastTapIn.csv', 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\5\\backtrace\\20221105-07-15-12-FastTapIn.csv', 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\6\\backtrace\\20221105-07-20-51-FastTapIn.csv', 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\7\\backtrace\\20221105-07-27-57-FastTapIn.csv', 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\8\\backtrace\\20221105-07-30-57-FastTapIn.csv', 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\9\\backtrace\\20221105-07-34-23-FastTapIn.csv'

In [253]:
import shutil
StayDestination = r"C:\Users\angel\Desktop\S1DA\veloAnalysation\StayVelo"
BacktraceDestination = r"C:\Users\angel\Desktop\S1DA\veloAnalysation\BacktraceVelo"

for item in stay_csvs:
    shutil.copy(item, StayDestination)

for item in backtrace_csvs:
    shutil.copy(item, BacktraceDestination)

In [267]:
des = BacktraceDestination

for file in os.listdir(des):
    if file.endswith("FastTapIn.csv"):
        newFilename = file.replace("FastTapIn.csv", "VelocityIn.csv")
        os.rename(os.path.join(des,file), os.path.join(des,newFilename))
        

In [23]:
backtrace_csvs

NameError: name 'backtrace_csvs' is not defined

In [236]:
stay_csvs


['C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\1\\stay\\20221105-06-47-21-FastTapIn.csv',
 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\10\\stay\\20230422-11-27-02-FastTapIn.csv',
 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\2\\stay\\20221105-06-58-18-FastTapIn.csv',
 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\3\\stay\\20221105-07-03-38-FastTapIn.csv',
 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\4\\stay\\20221105-07-10-56-FastTapIn.csv',
 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\5\\stay\\20221105-07-13-50-FastTapIn.csv',
 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\6\\stay\\20221105-07-19-39-FastTapIn.csv',
 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\7\\stay\\20221105-07-27-01-FastTapIn.csv',
 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\8\\stay\\20221105-07-30-04-FastTapIn.csv',
 'C:\\Users\\angel\\Desktop\\S1DA\\BacktraceDataset\\9\\stay\\20221105-07-33-29-FastTapIn.csv']

In [231]:
string = "20230422-11-28-31-AccelerationIn"
TestDate = string[2:8]

In [175]:
Folder_dir = r"C:\Users\angel\Desktop\S1DA\veloAnalysation"
StayFolder_dir = r"\StayVelo"
BacktrackFolder_dir =r"\BacktraceVelo"

Parent_folder_dir_stay = Folder_dir + StayFolder_dir
Parent_folder_dir_Backtrace = Folder_dir + BacktrackFolder_dir

if not(os.path.exists(Parent_folder_dir_stay)):
    os.makedirs(Parent_folder_dir_stay)
else:
    print("Path Exist")
if not(os.path.exists(Parent_folder_dir_Backtrace)):
    os.makedirs(Parent_folder_dir_Backtrace)
else:
    print("Path Exist")

Path Exist


In [184]:
folder_suffix =  " - Cancel"

for folder in os.listdir(Folder_dir):
    print(folder)
    for i in range(1,11):
        Folder_name = f"{i}{folder_suffix}"
        dir_path = os.path.join(Folder_dir, folder)
        subfolderPath = os.path.join(dir_path, Folder_name)
        if not(os.path.exists(subfolderPath)):
            os.makedirs(subfolderPath)
            print(f"{subfolderPath} created")
        else:
            print(f"{subfolderPath} exists")

BacktraceVelo
C:\Users\angel\Desktop\S1DA\veloAnalysation\BacktraceVelo\1 - Execute exists
C:\Users\angel\Desktop\S1DA\veloAnalysation\BacktraceVelo\2 - Execute exists
C:\Users\angel\Desktop\S1DA\veloAnalysation\BacktraceVelo\3 - Execute exists
C:\Users\angel\Desktop\S1DA\veloAnalysation\BacktraceVelo\4 - Execute exists
C:\Users\angel\Desktop\S1DA\veloAnalysation\BacktraceVelo\5 - Execute exists
C:\Users\angel\Desktop\S1DA\veloAnalysation\BacktraceVelo\6 - Execute exists
C:\Users\angel\Desktop\S1DA\veloAnalysation\BacktraceVelo\7 - Execute exists
C:\Users\angel\Desktop\S1DA\veloAnalysation\BacktraceVelo\8 - Execute exists
C:\Users\angel\Desktop\S1DA\veloAnalysation\BacktraceVelo\9 - Execute exists
C:\Users\angel\Desktop\S1DA\veloAnalysation\BacktraceVelo\10 - Execute exists
StayVelo
C:\Users\angel\Desktop\S1DA\veloAnalysation\StayVelo\1 - Execute exists
C:\Users\angel\Desktop\S1DA\veloAnalysation\StayVelo\2 - Execute exists
C:\Users\angel\Desktop\S1DA\veloAnalysation\StayVelo\3 - Execu

In [61]:
import datetime

workingPath = r"C:\Users\angel\Desktop\S1DA"
study_num = 1
today = str(datetime.date.today()).replace("-","")[2:]

folder_name = f"{TestDate} - study{study_num}"
print(f"root_dir --> {folder_name}")

root_dir --> 230422 - study1


In [14]:
directory = r"C:\Users\angel\Desktop\S1DA\Separated - Backtrace - 230422 - study1\Cancel"

In [15]:
# File Paths
root_dir = directory
analyze_date = "221212"
out_dir = root_dir + " - out"
execute_dir = []
cancel_dir = []
if not os.path.exists(out_dir):
  os.makedirs(out_dir)
for dir in os.listdir(root_dir):
  if dir.endswith("Cancel"):
    cancel_dir.append(dir)
  elif dir.endswith("Execute"):
    execute_dir.append(dir)
long_in_csvs = []
fast_in_csvs = []
velo_in_csvs = []
acce_in_csvs = []
for dir in execute_dir:
  temp = root_dir+"/"+dir
  for csv in os.listdir(temp):
    temp_csv = temp + "/" + csv
    if csv.endswith("LongTapIn.csv"):
      long_in_csvs.append(temp_csv)
    elif csv.endswith("FastTapIn.csv"):
      fast_in_csvs.append(temp_csv)
    elif csv.endswith("VelocityIn.csv"):
      velo_in_csvs.append(temp_csv)
    elif csv.endswith("AccelerationIn.csv"):
      acce_in_csvs.append(temp_csv)
print("LongTapInExecute:",long_in_csvs)
print("FastTapInExecute:",fast_in_csvs)
print("VelocityInExecute:",velo_in_csvs)
print("AcceInExecute:",acce_in_csvs)

LongTapInExecute: []
FastTapInExecute: []
VelocityInExecute: []
AcceInExecute: []


In [275]:
# File Paths
root_dir = "221209 - study1"
analyze_date = "221212"
out_dir = root_dir + " - out"
execute_dir = []
cancel_dir = []
if not os.path.exists(out_dir):
  os.makedirs(out_dir)
for dir in os.listdir(root_dir):
  if dir.endswith("Cancel"):
    cancel_dir.append(dir)
  elif dir.endswith("Execute"):
    execute_dir.append(dir)
long_in_csvs = []
fast_in_csvs = []
velo_in_csvs = []
acce_in_csvs = []
for dir in execute_dir:
  temp = root_dir+"/"+dir
  for csv in os.listdir(temp):
    temp_csv = temp + "/" + csv
    if csv.endswith("LongTapIn.csv"):
      long_in_csvs.append(temp_csv)
    elif csv.endswith("FastTapIn.csv"):
      fast_in_csvs.append(temp_csv)
    elif csv.endswith("VelocityIn.csv"):
      velo_in_csvs.append(temp_csv)
    elif csv.endswith("AccelerationIn.csv"):
      acce_in_csvs.append(temp_csv)
print("LongTapInExecute:",long_in_csvs)
print("FastTapInExecute:",fast_in_csvs)
print("VelocityInExecute:",velo_in_csvs)
print("AcceInExecute:",acce_in_csvs)

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: '221209 - study1'

In [16]:
# LongTapTime
'''Output: Number (Subject Number) | Case (Enter Case Number) | EnterTime (Selected Biggest EnterTime)'''
long_save_name = analyze_date + ' - LongTapTime.csv'
long_df = pd.DataFrame(columns=['Number','Case','EnterTime'])
for csv in long_in_csvs:
  index = csv.split('/')[1].split(' - ')[1]
  df = get_biggest_EnterTime(csv, 'Case', 'EnterTime')
  df.insert(df.shape[1], 'Number', index)
  long_df = pd.concat([long_df,df])
long_df = remove_outliers(long_df, 'EnterTime', False)
long_df.to_csv(out_dir + '/' + long_save_name, index=False)
print("Average Biggest Enter Time for Long Tap:",long_df['EnterTime'].mean())

Average Biggest Enter Time for Long Tap: nan


In [17]:
# FastTapTime
'''Output: Number (Subject Number) | Case (Enter Case Number) | EnterTime (Selected Biggest EnterTime)'''
fast_save_name = analyze_date + ' - FastTapTime.csv'
fast_df = pd.DataFrame(columns=['Number','Case','EnterTime'])
for csv in fast_in_csvs:
  index = csv.split('/')[1].split(' - ')[1]
  df = get_biggest_EnterTime(csv, 'Case', 'EnterTime')
  df.insert(df.shape[1], 'Number', index)
  fast_df = pd.concat([fast_df,df])
fast_df = remove_outliers(fast_df, 'EnterTime', False)
fast_df.to_csv(out_dir + '/' + fast_save_name, index=False)
print("Average Biggest Enter Time for Fast Tap:",fast_df['EnterTime'].mean())

Average Biggest Enter Time for Fast Tap: nan


In [18]:
# Velocity
velo_df = pd.DataFrame(columns=['Number','Case','Velocity'])
for csv in velo_in_csvs:
  index = csv.split('/')[1].split(' - ')[1]
  df = pd.read_csv(csv)
  df = df[['Case','Velocity']]
  df = remove_outliers(df, 'Velocity', True)
  df.insert(df.shape[1], 'Number', index)
  velo_df = pd.concat([velo_df,df])
velo_df_grp = velo_df.groupby(['Number','Case'])
velo_by_case = describe_by_Case(velo_df_grp,'Velocity')
velo_min_mean = velo_by_case['Min'].mean()
velo_mean_mean = velo_by_case['Mean'].mean()
velo_max_mean = velo_by_case['Max'].mean()
velo_df.to_csv(out_dir + '/' + analyze_date + ' - Velo_no_outliers.csv', index=False)
velo_by_case.to_csv(out_dir + '/' + analyze_date + ' - Velo_by_case.csv', index=False)
velo_dist = (velo_mean_mean - velo_min_mean)/3
print("Velocity for Button Number 4's condition A:",velo_mean_mean)
print("Velocity for condition C:", velo_max_mean)
print("Velocity distance:", velo_dist)
print("The Velocity of condition A for the seven models:")
for i in range(-3,4):
  print(velo_mean_mean + velo_dist * i)

Velocity for Button Number 4's condition A: 0.5465419988834879
Velocity for condition C: 0.8620274462962964
Velocity distance: 0.07943817378832313
The Velocity of condition A for the seven models:
0.3082274775185185
0.3876656513068416
0.46710382509516474
0.5465419988834879
0.625980172671811
0.7054183464601341
0.7848565202484572


In [19]:
# Acceleration
acce_df = pd.DataFrame(columns=['Number','Case','Acceleration'])
for csv in acce_in_csvs:
  print(csv)
  index = csv.split('/')[1].split(' - ')[1]
  print(f"Index is {index}")

  df = pd.read_csv(csv)
  df = df[['Case','Acceleration']]
  df = df[df['Acceleration']<0]
  df = remove_outliers(df, 'Acceleration', True)
  df.insert(df.shape[1], 'Number', index)
  acce_df = pd.concat([acce_df,df])
acce_df_grp = acce_df.groupby(['Number','Case'])
acce_by_case = describe_by_Case(acce_df_grp,'Acceleration')
acce_min_mean = acce_by_case['Min'].mean()
acce_mean_mean = acce_by_case['Mean'].mean()
acce_max_mean = acce_by_case['Max'].mean()
acce_df.to_csv(out_dir + '/' + analyze_date + ' - Acce_no_outliers.csv', index=False)
acce_by_case.to_csv(out_dir + '/' + analyze_date + ' - Acce_by_case.csv', index=False)
acce_dist = (acce_mean_mean - acce_max_mean)/3
print("Acceleration for Button Number 4's condition B:",acce_mean_mean)
print("Acceleration distance:",acce_dist)
print("The Acceleration for the seven models:")
for i in range(-3,4):
  print(acce_mean_mean + acce_dist * i)

C:\Users\angel\Desktop\S1DA\Test_pipelline\test\230110 - cancel - study1/230110 - 1 - Execute/230110 - 1- AccelerationIn.csv
Index is 1
C:\Users\angel\Desktop\S1DA\Test_pipelline\test\230110 - cancel - study1/230110 - 2 - Execute/230110 - 2- AccelerationIn.csv
Index is 2
C:\Users\angel\Desktop\S1DA\Test_pipelline\test\230110 - cancel - study1/230110 - 3 - Execute/230110 - 3- AccelerationIn.csv
Index is 3
C:\Users\angel\Desktop\S1DA\Test_pipelline\test\230110 - cancel - study1/230110 - 4 - Execute/230110 - 4- AccelerationIn.csv
Index is 4
Acceleration for Button Number 4's condition B: -17.86626685909142
Acceleration distance: -2.536570384445568
The Acceleration for the seven models:
-10.256555705754717
-12.793126090200285
-15.329696474645853
-17.86626685909142
-20.40283724353699
-22.939407627982558
-25.475978012428126


In [120]:
csv = r'C:\Users\angel\Desktop\S1DA\230422 - study1/1- Execute/20221105-06-47-21-AccelerationIn.csv'

index = csv.split('/')[1].split(' - ')[1]

Execute


In [21]:
print(f"mean =  {acce_mean_mean}, min mean = {acce_min_mean}, max mean = {acce_max_mean}")

mean =  -11.033983503874563, min mean = -86.984883075, max mean = -1.4309683962725
